In [44]:
import json
import requests
import pandas as pd
import numpy as np

import sqlite3
from uszipcode import ZipcodeSearchEngine

In [45]:
conn = sqlite3.connect('raw_data_no_lat_lon.sqlite')

# us_data_df = pd.read_sql("department_contracts", conn)


us_data_df = pd.read_sql_query("SELECT * FROM department_contracts", conn)

In [46]:
us_data_df.head()


,index,Awarding_Agency,Category,Contract_ID,Description,POP_City,POP_State,POP_Zip,Recipient_Name,Subtier_Agency,Subtier_Code,Total_Obligation
0,0,Department of Transportation,contract,17543,None,WASHINGTON,DISTRICT OF COLUMBIA,20591,EVANS INCORPORATED,Immediate Office of the Secretary of Transport...,6901,745068.00
1,1,Department of Transportation,contract,31520,None,BALTIMORE,MARYLAND,21230,MARYLAND MARITIME INC,Maritime Administration,6938,520169.00
2,2,Department of Transportation,contract,31521,None,BALTIMORE,MARYLAND,21230,MARYLAND MARITIME INC,Maritime Administration,6938,662420.00
3,3,Department of Transportation,contract,31525,None,ORLANDO,FLORIDA,32822,"SIMCOM INTERNATIONAL, INC.",Federal Aviation Administration,6920,39520.00
4,4,Department of Transportation,contract,31526,None,MOJAVE,CALIFORNIA,93501,NATIONAL TEST PILOT SCHOOL INC,Federal Aviation Administration,6920,29950.00


In [47]:
zip_code = pd.DataFrame(us_data_df['POP_Zip'].unique())
zip_code['Latitude'] = ''
zip_code['Longitude'] = ''
zip_code.columns = [
    'POP_Zip',
    'Latitude',
    'Longitude'
]

zip_code.head()



,POP_Zip,Latitude,Longitude
0,20591,,
1,21230,,
2,32822,,
3,93501,,
4,11371,,


In [48]:
# test = {'Zip':['94102', '21230','32822','93850','11371'],
#         'Latitude':['','','','',''],
#         'Longitude':['','','','','']}
# test_df = pd.DataFrame(test)
# test_df

In [49]:

search = ZipcodeSearchEngine()

for index, row in zip_code.iterrows():    
    zipcode = search.by_zipcode(row['POP_Zip']) 
    row['Latitude'] = zipcode['Latitude']
    row['Longitude'] = zipcode['Longitude'] 
    
# for index, row in us_data_df.iterrows():    
#     if row['Latitude'] == None:
#         res = search.by_city(row['POP_City'])
#         row['Latitude']= res[0]['Latitude']
#         row['Longitude'] = res[0]['Longitude']

zip_code.head()

,POP_Zip,Latitude,Longitude
0,20591,None,None
1,21230,39.2635,-76.6239
2,32822,28.475,-81.2849
3,93501,35.044,-118.253
4,11371,40.7766,-73.8743


In [50]:
# us_data_df['Latitude'] = ''
# us_data_df['Longitude'] = ''
us_data_df.head()

,index,Awarding_Agency,Category,Contract_ID,Description,POP_City,POP_State,POP_Zip,Recipient_Name,Subtier_Agency,Subtier_Code,Total_Obligation
0,0,Department of Transportation,contract,17543,None,WASHINGTON,DISTRICT OF COLUMBIA,20591,EVANS INCORPORATED,Immediate Office of the Secretary of Transport...,6901,745068.00
1,1,Department of Transportation,contract,31520,None,BALTIMORE,MARYLAND,21230,MARYLAND MARITIME INC,Maritime Administration,6938,520169.00
2,2,Department of Transportation,contract,31521,None,BALTIMORE,MARYLAND,21230,MARYLAND MARITIME INC,Maritime Administration,6938,662420.00
3,3,Department of Transportation,contract,31525,None,ORLANDO,FLORIDA,32822,"SIMCOM INTERNATIONAL, INC.",Federal Aviation Administration,6920,39520.00
4,4,Department of Transportation,contract,31526,None,MOJAVE,CALIFORNIA,93501,NATIONAL TEST PILOT SCHOOL INC,Federal Aviation Administration,6920,29950.00


In [51]:
# search = ZipcodeSearchEngine()

# for index, row in us_data_df.iterrows():    
#     zipcode = search.by_zipcode(row['POP_Zip'])
#     zip_lat = zipcode['Latitude']
#     zip_lon = zipcode['Longitude']
    
#     if zip_lat == None:
#         res = search.by_city(row['POP_City'])
#         try:
#             row['Latitude']= res[0]['Latitude']
#         except:
#             row['Latitude']= None
#         try:
#             row['Longitude']= res[0]['Longitude']
#         except:
#             row['Longitude']= None
#     else:       
#         row['Latitude'] = zip_lat
#         row['Longitude'] = zip_lon
    
# for index, row in us_data_df.iterrows():    
#     if row['Latitude'] == None:
#         res = search.by_city(row['POP_City'])
#         row['Latitude']= res[0]['Latitude']
# #         row['Longitude'] = res[0]['Longitude']

# conn = sqlite3.connect('final_dataset.sqlite')

# us_data_df.to_sql("department_contracts", conn, if_exists="replace")

In [52]:
merged_df = us_data_df.merge(zip_code, how='left', on='POP_Zip')

merged_df = merged_df[pd.notnull(merged_df['Latitude'])]
merged_df
# zip_code.head()

,index,Awarding_Agency,Category,Contract_ID,Description,POP_City,POP_State,POP_Zip,Recipient_Name,Subtier_Agency,Subtier_Code,Total_Obligation,Latitude,Longitude
1,1,Department of Transportation,contract,31520,None,BALTIMORE,MARYLAND,21230,MARYLAND MARITIME INC,Maritime Administration,6938,520169.00,39.2635,-76.6239
2,2,Department of Transportation,contract,31521,None,BALTIMORE,MARYLAND,21230,MARYLAND MARITIME INC,Maritime Administration,6938,662420.00,39.2635,-76.6239
3,3,Department of Transportation,contract,31525,None,ORLANDO,FLORIDA,32822,"SIMCOM INTERNATIONAL, INC.",Federal Aviation Administration,6920,39520.00,28.475,-81.2849
4,4,Department of Transportation,contract,31526,None,MOJAVE,CALIFORNIA,93501,NATIONAL TEST PILOT SCHOOL INC,Federal Aviation Administration,6920,29950.00,35.044,-118.253
5,5,Department of Transportation,contract,31527,None,FLUSHING,NEW YORK,11371,FLIGHTSAFETY INTERNATIONAL INC,Federal Aviation Administration,6920,59200.00,40.7766,-73.8743
7,7,Department of Transportation,contract,31534,None,SAINT PETERSBURG,FLORIDA,33710,"SENSOR SYSTEMS, L.L.C.",Federal Aviation Administration,6920,211847.00,27.792,-82.7239
8,8,Department of Transportation,contract,31535,None,OKLAHOMA CITY,OKLAHOMA,73169,ONESIMUS DEFENSE LLC,Federal Aviation Administration,6920,13989.00,35.3787,-97.644
10,10,Department of Transportation,contract,31537,None,TUSTIN,CALIFORNIA,92780,"PVP ADVANCED EO SYSTEMS, INC.",Federal Aviation Administration,6920,106500.00,33.7372,-117.814
11,11,Department of Transportation,contract,31538,None,OKLAHOMA CITY,OKLAHOMA,73169,RIGIL CORPORATION,Federal Aviation Administration,6920,605377.86,35.3787,-97.644
12,12,Department of Transportation,contract,31539,None,COLORADO SPRINGS,COLORADO,80916,"RAMPART AVIATION, LLC",Federal Aviation Administration,6920,15830.00,38.8022,-104.723
